In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# 임베딩 모델로 텍스트 수치화
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')
embeddings = embeddings_model.embed_documents(
    ["Hi there!", "Oh hello!", "What's your name?", "My friends call me World", "Hello World!"]
)

len(embeddings), len(embeddings[0]),
print(embeddings[0] [: 10])

[-0.019139237701892853, -0.03814302384853363, -0.03093702718615532, -0.004656130913645029, -0.03535273298621178, -0.003945012576878071, 0.013010076247155666, 0.05103796720504761, -0.005804079119116068, -0.0371948666870594]


In [3]:
# pip install pypdf
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# PDF 문서 로드
loader = PyPDFLoader(r"../content/202408_이력서.pdf")
pages = loader.load()

# PDF 문서를 여러 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

texts = text_splitter.split_documents(pages)

# 임베딩 모델 API 호출
embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')

# OpenAI 임베딩 모델로 청크들을 임베딩 변환하기
embeddings = embeddings_model.embed_documents([i.page_content for i in texts])
len(embeddings), len(embeddings[0]),

ValueError: File path ../content/202408_이력서.pdf is not a valid file or url

In [15]:
# vector store
# pip install faiss-cpu
from langchain.vectorstores import FAISS

embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')

# PDF 문서를 여러 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

texts = text_splitter.split_documents(pages)

db = FAISS.from_documents(texts ,embeddings_model)

# Retriever 생성
retriever = db.as_retriever()
query = "가장 최근 작업이 뭐야?"
#유사 문서 검색
result = retriever.invoke(query)
content = result.pop().page_content
content

NameError: name 'pages' is not defined

In [22]:
# pdf 내용이 저장된 벡터 스토어에서 조회를 통하여 질문에 답을 해보는 langChain
from langchain_openai import ChatOpenAI
from langchain import hub # langchain 커뮤니티에서 프롬포트 공유 플랫폼 : 내가 찾고자하는 프롬포트를 검색해서 그대로 pull 해주는 라이브러리
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = ChatOpenAI(model="gpt-4o-mini")
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(docs.page_content for docs in docs) # Document 객체에있는 page_content 만 이어붙인다.

rag_chain = (
    {"context" : retriever | format_docs, "question" : RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("내 경력에서 자바에서 코틀린 변환 작업에 대해서 설명해줄래?")


'지금까지 자바에서 코틀린으로의 변환 작업에 참여한 경험이 있습니다. 이 과정에서 NPE 문제 해결과 함께 코드를 최적화하여 성능 개선을 이루었습니다. 변환된 코드에서 27.7%의 성능 향상을 달성하였습니다.'

In [4]:
# langgraph with tool
# llm.bind_tools(tool) : 모델이 활용할 수 있는 툴을 바인딩
# Conditional edge : tool 필요없이 모델이 직접 답할 수 있는 경우가 있기 때문에 조건에 따라 호출할 수 있도록 한다.
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import AIMessage

from langgraph.prebuilt import ToolNode

# 툴을 정의하기
@tool # tool ㅇㅣ용할 때 사용하는 데코레이터
def get_weather(location :str):
    """Call to get the weather""" # 에이전트가 어떤 도구를 사용할 지 선택할 때 사용하는 힌트를 줄 수 있겠지
    if location in ["대구", "전주"]:
        return "ㄷㅓ워 주거요"
    else:
        return "추워 주거요"


@tool
def get_coolest_cities():
    """Call to get the coolest cities"""
    return "강릉"

In [5]:
# 툴 노드 세팅
tools = [get_weather, get_coolest_cities]
tool_node = ToolNode(tools)

In [6]:
# 모델이 툴 바인딩
model_with_tools = (ChatOpenAI(model="gpt-4o-mini", temperature = 0).bind_tools(tools))

In [16]:
# 모델에 질문하기
model_with_tools.invoke("전주 날씨 알려줄래?").tool_calls

[{'name': 'get_weather',
  'args': {'location': '전주'},
  'id': 'call_ggOiyeRAAkgs9mnfMTUzJi7H',
  'type': 'tool_call'}]

In [8]:
# 두 개의 도구로 활용 및 답변할 수 없다면, 아무런 답도 없음
model_with_tools.invoke("이 세상에서 누가 제일 예쁘니?").tool_calls

[]

In [9]:
# 이때 적용되는 get_coolest_cities 함수는 인자로 location 넘겨주지 않기 때문에 args 가 비어있다.
model_with_tools.invoke("제일 추운 도시는?").tool_calls

[{'name': 'get_coolest_cities',
  'args': {},
  'id': 'call_ku3XrecznbbMbXLbyCgtJccB',
  'type': 'tool_call'}]

In [17]:
# ai 의 요청을 tool 이 실제로 실행한 결과
tool_node.invoke({"messages": [model_with_tools.invoke("전주 날씨는 어때?")]})

{'messages': [ToolMessage(content='ㄷㅓ워 주거요', name='get_weather', tool_call_id='call_3giUWyRGIjTZoVRz5pwzYPud')]}

In [22]:
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import HumanMessage
from langgraph.graph import END, START, StateGraph, MessagesState

# 아래 conditional_edge 는 should_continue 라는 함수에 따라 agent 노드에서 tool 혹은 end로 진입시킵니다.
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls: # 질문이 도구를 불러야되는 거였다면
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END

def call_model(state: MessagesState):
    messages = state['messages']
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model) #model 이 실행하는 함수
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

# add_conditional_edges는 조건부 엣지 추가 함수
workflow.add_conditional_edges(
    "agent", # 시작점
    should_continue, #시작점의 응답으로 호출할 함수
)

workflow.add_edge("tools", 'agent')

app = workflow.compile()

In [20]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="전주의 날씨는 어때?")]}
)
final_state["messages"][-1].content

'전주의 날씨는 덥습니다. 더 구체적인 정보가 필요하시면 말씀해 주세요!'

In [21]:
# example with a multiple tool calls in succession

for chunk in app.stream(
    {"messages": [("human", "가장 추운 도시의 날씨는 어때?")]},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

가장 추운 도시의 날씨는 어때?
================================== Ai Message ==================================
Tool Calls:
  get_coolest_cities (call_Da0VJSmdDf1wESW7UfPNjtZI)
 Call ID: call_Da0VJSmdDf1wESW7UfPNjtZI
  Args:
================================= Tool Message =================================
Name: get_coolest_cities

강릉
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_gnzL8Z5nnqw3ANJnykvCtauq)
 Call ID: call_gnzL8Z5nnqw3ANJnykvCtauq
  Args:
    location: 강릉
================================= Tool Message =================================
Name: get_weather

추워 주거요
================================== Ai Message ==================================

강릉의 날씨는 매우 춥습니다. 추가적인 정보가 필요하시면 말씀해 주세요!
